In [1]:
import pandas as pd
import torch

In [2]:
LANGUAGE = "rus_Cyrl"

In [3]:
# Run sample_1m.py to generate the samples
samples = pd.read_csv(f"samples-{LANGUAGE}-1m-200_set.csv").drop(columns=["Unnamed: 0"])
samples = samples[["id", "u", "ts", "text"]]

In [4]:
samples

,id,u,ts,text
0,5e8594581fc85baed81426cec02803bb,http://booksshare.net/index.php?id1=4&category...,2017-06-28T05:53:48Z,booksshare.net -> Добавить материал -> Химия\n...
1,fa725365771cb33d45cafdca2d5f589c,http://patents.su/metka/soderzhaniya/page/31,2017-08-03T21:19:23Z,Патенты с меткой «содержания»\nСпособ измерени...
2,b9a4b0b3ad80fdd6b3d4e9c343f69967,http://www.pressreader.com/ukraine/football-uk...,2018-01-19T11:57:54Z,"ДОЛГОВАЯ «ГАБАЛА»\n«ШАХТЕР»\nДончане, успевшие..."
3,a5e77faa7c8e7b5cadbc261ecee5a410,http://spec-park.ru/products/yamobur-spb,2017-04-16T01:46:03Z,Аренда ямобура БКМ AICHI 902 на базе Камаз (ве...
4,a35e313adccf962717fd7221b0fbba6d,http://gazetarb.ru/columns/author-utaliev/arti...,2017-09-26T10:52:40Z,Быдлоград\nКолумнист: Тимур Уталиев\n- 9 Сентя...
...,...,...,...,...
195,5e26bd5afa2a794a33d72306a70aca9e,https://azimutnt.ru/2014/%D0%BC%D0%B0%D0%B9%D1...,2022-09-29T20:29:04Z,Майская вело-прогулка\nПо весеннему тёплой пог...
196,def153514bd655811586ace1402e848b,https://ru.reuters.com/article/marketsNews/idR...,2018-01-17T22:11:20Z,"АЛМА-АТА, 28 июл (Рейтер) - Казахстанская нефт..."
197,8d32c9bdec8b07294b469a1f009770db,http://sweet211.ru/rosnedra-vistavili-na-torgi...,2017-08-30T17:03:47Z,Роснедра планируют провести 24 сентября 2014 г...
198,cf0c1fb54738485be98361715e3a8c0e,http://isfic.info/pikul/gosupr48.htm,2017-09-24T02:59:18Z,Управление нефтяной отраслью\nХарактерная особ...


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "TurkuNLP/multilingual-web-register-classification"

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

def infer_text(text):
    # Tokenize text
    inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Apply sigmoid to the logits to get probabilities
    probabilities = torch.sigmoid(outputs.logits).squeeze()

    # Determine a threshold for predicting labels
    threshold = 0.5
    predicted_label_indices = (probabilities > threshold).nonzero(as_tuple=True)[0]

    # # Extract readable labels using id2label
    id2label = model.config.id2label
    predicted_labels = [id2label[idx.item()] for idx in predicted_label_indices]
    return predicted_labels


/home/egor/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from tqdm import tqdm

labels = [infer_text(record) for record in tqdm(samples['text'])]

100%|██████████| 200/200 [00:45<00:00,  4.43it/s]


In [7]:
samples_with_labels = samples.copy()
samples_with_labels["labels"] = [", ".join(label) for label in labels]

In [10]:
samples_with_labels["gen_label"] = samples_with_labels["labels"].apply(
    lambda x: x[:2]
)

In [15]:
samples_with_labels["gen_label"].value_counts() / len(samples_with_labels)

gen_label
NA    0.325
IN    0.285
IP    0.190
MT    0.085
OP    0.030
ID    0.030
      0.030
SP    0.015
HI    0.010
Name: count, dtype: float64

In [9]:
samples_with_labels.to_csv(f"samples-with-labels-{LANGUAGE}-1m-200_set.csv")